In [12]:
from PIL import Image
import IPython.display as IPdisplay
import matplotlib.pyplot as plt
import glob
import os
import pickle
%matplotlib widget


In [13]:
def get_Lorenz_attractor(iterations):
    x, y, z = 0.01, 0, 0
    sigma, rho, beta = 10, 28, 8 / 3
    dt = 0.01
    result = [[x], [y], [z]]
    for i in range(0, iterations):
        dx = (sigma * (y - x)) * dt
        dy = (x * (rho - z) - y) * dt
        dz = (x * y - beta * z) * dt
        x += dx
        y += dy
        z += dz
        result[0].append(x)
        result[1].append(y)
        result[2].append(z)
    return result


In [14]:
def get_Aizawa_attractor(iterations):
    x, y, z = 0.1, 0, 0
    a, b, c, d, e, f = 0.95, 0.7, 0.6, 3.5, 0.25, 0.1
    dt = 0.01
    result = [[x], [y], [z]]
    for i in range(0, iterations):
        dx = (z - b) * x - d * y
        dy = d * x + (z - b) * y
        dz = c + a * z - (pow(z, 3) / 3) - (pow(x, 2) +
                                            pow(y, 2)) * (1 + e * z) + f * z * (pow(x, 3))
        x += dx * dt
        y += dy * dt
        z += dz * dt
        result[0].append(x)
        result[1].append(y)
        result[2].append(z)
    return result


In [15]:
def adjust_scale(bounds, axes_shift, delta):
    max_ind, max_val = 0, delta[0]
    for i, val in enumerate(delta[1:]):
        if val > max_val:
            max_ind, max_val = i + 1, val
    for i in range(0, len(bounds)):
        if i == max_ind:
            bounds[i][0] -= axes_shift * delta[i]
            bounds[i][1] += axes_shift * delta[i]
        else:
            bounds[i][0] -= (delta[max_ind] *
                             (1 + axes_shift * 2) - delta[i]) / 2
            bounds[i][1] += (delta[max_ind] *
                             (1 + axes_shift * 2) - delta[i]) / 2


In [16]:
def generate_animation_2d(x_arr, y_arr, frames_number, frame_duration, file_name, darkmode=True):
    if darkmode:
        plt.style.use('dark_background')
    else:
        plt.style.use('default')
    x_min = min(x_arr)
    x_max = max(x_arr)
    y_min = min(y_arr)
    y_max = max(y_arr)
    dx = x_max - x_min
    dy = y_max - y_min
    axes_shift = 0.1
    bounds = [[x_min, x_max], [y_min, y_max]]
    delta = [dx, dy]
    adjust_scale(bounds, axes_shift, delta)

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)

    ax.set_xlim(bounds[0][0], bounds[0][1])
    ax.set_ylim(bounds[1][0], bounds[1][1])

    save_folder = file_name + '_images'
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    else:
        for f in os.listdir(save_folder):
            os.remove(os.path.join(save_folder, f))
    points_number = min(len(x_arr), len(y_arr))
    frame_perioud = round(points_number / frames_number)
    if points_number < frames_number:
        frame_perioud = 1
        frames_number = points_number
    for i in range(1, points_number - 1, frame_perioud):
        ax.plot(x_arr[i - 1: i + frame_perioud], y_arr[i - 1: i + frame_perioud], c='#65aeeb', linewidth=0.6)
        plt.savefig('{}/{:03d}.png'.format(save_folder, round(i / frame_perioud)), dpi=150, bbox_inches='tight', pad_inches=0.1)
    plt.savefig('{}/{:03d}.png'.format(save_folder, round(points_number / frame_perioud)), dpi=150, bbox_inches='tight', pad_inches=0.1)
                
    images = [Image.open(image) for image in glob.glob('{}/*.png'.format(save_folder))]
    gif_filepath = file_name + ".gif"

    duration = frame_duration * frames_number
    gif = images[0]
    gif.info['duration'] = duration
    gif.info['loop'] = 0
    gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images)

    plt.show()


In [17]:
def generate_animation_3d(x_arr, y_arr, z_arr, frames_number, frame_duration, file_name, with_projections=False, darkmode=True):
    if darkmode:
        plt.style.use('dark_background')
    else:
        plt.style.use('default')
    x_min = min(x_arr)
    x_max = max(x_arr)
    y_min = min(y_arr)
    y_max = max(y_arr)
    z_min = min(z_arr)
    z_max = max(z_arr)
    dx = x_max - x_min
    dy = y_max - y_min
    dz = z_max - z_min
    axes_shift = 0.1
    bounds = [[x_min, x_max], [y_min, y_max], [z_min, z_max]]
    delta = [dx, dy, dz]
    adjust_scale(bounds, axes_shift, delta)

    points_number = min(len(x_arr), len(y_arr), len(z_arr))

    fig = plt.figure()
    line_width = 0.9
    if with_projections:
        ax_3d = fig.add_subplot(2, 2, 1, projection='3d')
        ax_3d.set_xlabel('x')
        ax_3d.set_ylabel('y')
        ax_3d.set_zlabel('z', rotation=0, labelpad=2)
        ax1 = fig.add_subplot(2, 2, 2)
        ax1.set_xlim((bounds[0][0], bounds[0][1]))
        ax1.set_ylim((bounds[1][0], bounds[1][1]))
        ax1.set_xlabel('x')
        ax1.set_ylabel('y', rotation=0, labelpad=5)
        ax2 = fig.add_subplot(2, 2, 3)
        ax2.set_xlim((bounds[0][0], bounds[0][1]))
        ax2.set_ylim((bounds[2][0], bounds[2][1]))
        ax2.set_xlabel('x')
        ax2.set_ylabel('z', rotation=0, labelpad=5)
        ax3 = fig.add_subplot(2, 2, 4)
        ax3.set_xlim((bounds[1][0], bounds[1][1]))
        ax3.set_ylim((bounds[2][0], bounds[2][1]))
        ax3.set_xlabel('y')
        ax3.set_ylabel('z', rotation=0, labelpad=5)
        line_width = 0.6
        plt.tight_layout()
    else:
        ax_3d = fig.add_subplot(projection='3d')

    ax_3d.set_xlim((bounds[0][0], bounds[0][1]))
    ax_3d.set_ylim((bounds[1][0], bounds[1][1]))
    ax_3d.set_zlim((bounds[2][0], bounds[2][1]))
    if darkmode:
        ax_3d.xaxis.set_pane_color((0.07, 0.07, 0.07, 1.0))
        ax_3d.yaxis.set_pane_color((0.07, 0.07, 0.07, 1.0))
        ax_3d.zaxis.set_pane_color((0.07, 0.07, 0.07, 1.0))
        ax_3d.xaxis._axinfo["grid"].update({"color": (0.7, 0.7, 0.7, 1.0)})
        ax_3d.yaxis._axinfo["grid"].update({"color": (0.7, 0.7, 0.7, 1.0)})
        ax_3d.zaxis._axinfo["grid"].update({"color": (0.7, 0.7, 0.7, 1.0)})

    save_folder = file_name + '_images'
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    else:
        for f in os.listdir(save_folder):
            os.remove(os.path.join(save_folder, f))
    frame_perioud = round(points_number / frames_number)
    if points_number < frames_number:
        frame_perioud = 1
        frames_number = points_number
    for i in range(0, points_number - 1, frame_perioud):
        ax_3d.plot(x_arr[i - 1: i + frame_perioud], y_arr[i - 1: i + frame_perioud],
                   z_arr[i - 1: i + frame_perioud], c='#65aeeb', linewidth=line_width)
        if with_projections:
            ax1.plot(x_arr[i - 1: i + frame_perioud], y_arr[i - 1: i + frame_perioud], c='#65aeeb', linewidth=line_width)
            ax2.plot(x_arr[i - 1: i + frame_perioud], z_arr[i - 1: i + frame_perioud], c='#65aeeb', linewidth=line_width)
            ax3.plot(y_arr[i - 1: i + frame_perioud], z_arr[i - 1: i + frame_perioud], c='#65aeeb', linewidth=line_width)
			
        plt.savefig('{}/{:03d}.png'.format(save_folder, round(i / frame_perioud)),
                    dpi=150, bbox_inches='tight', pad_inches=0.1)
    
    plt.savefig('{}/{:03d}.png'.format(save_folder, round(points_number / frame_perioud)),
                dpi=150, bbox_inches='tight', pad_inches=0.1)

    images = [Image.open(image)
              for image in glob.glob('{}/*.png'.format(save_folder))]
    gif_filepath = file_name + ".gif"

    duration = frame_duration * frames_number
    gif = images[0]
    gif.info['duration'] = duration
    gif.info['loop'] = 0
    gif.save(fp=gif_filepath, format='gif',
             save_all=True, append_images=images)

    plt.show()


In [ ]:
xyz = get_Lorenz_attractor(7000)
generate_animation_3d(xyz[0], xyz[1], xyz[2], 10, 0.04, "Lorenz")

In [19]:
Image.open("Lorenz.gif")
IPdisplay.Image(url="Lorenz.gif")

In [ ]:
xyz = get_Aizawa_attractor(10000)
generate_animation_3d(xyz[0], xyz[1], xyz[2], 200, 0.02, "Aizawa_p", True)

In [21]:
Image.open("Aizawa_p.gif")
IPdisplay.Image(url="Aizawa_p.gif")

In [22]:
# generate_animation_2d(xyz[0], xyz[1], 100, 0.2, "Aizawa2d")

# Image.open("Aizawa2d.gif")
# IPdisplay.Image(url="Aizawa2d.gif")
